# 프로젝트 가보

In [21]:
from pdb import post_mortem
from unittest import result
from flask import Flask, render_template, redirect, request, url_for, session
import cx_Oracle as db

app = Flask(__name__)

db_id = 'parky'
db_pw = 'qwer1234'
url = '192.168.35.90:1521/xe'

app.secret_key = "gabo"

    
#### 관리자 페이지
#로그인 페이지
@app.route('/', methods=['GET'])
def loginPage():
    return render_template('page-login.html')

#로그인 체크
@app.route('/loginCheck', methods = ['POST']) #로그인 기능
def loginCheck():
    print('로그인 요청')
    admin_id = request.form['id']
    admin_pw = request.form['pw']
    
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select * from t_admin where admin_id = :1 and admin_pw = :2"
    curs.execute(sql, (admin_id, admin_pw))
    result = curs.fetchall();

    curs.close()
    conn.close()
    print(admin_id)
    print(result[0][0])
    if result[0][0] ==  admin_id and result[0][1]==admin_pw:
        session['admin_id'] = admin_id
        return redirect(url_for('mainPage'))
    
# 회원가입 페이지
@app.route('/joinPage', methods=['GET'])
def joinPage():
    return render_template('page-join.html')


# 회원가입
@app.route('/joinAdmin', methods = ['POST'])
def joinAdmin():
    print('회원가입 요청')
    admin_id = request.form['id']
    admin_pw = request.form['pw']
    admin_name = request.form['name']
    print(admin_id)
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
#     sql = "insert into t_admin values('%s', '%s', '%s')" %(admin_id, admin_pw, admin_name)
    sql = "insert into t_admin values(:1, :2, :3)"
#     curs.execute(sql)
    curs.execute(sql,(admin_id,admin_pw,admin_name))

    conn.commit()
    curs.close()
    conn.close()
#     return redirect(url_for('/'))
    return render_template('page-login.html')

# 보물등록요청 리스트
@app.route('/tList', methods = ['GET'])
def tList():
    #login_user = session['login_user']
    print('등록리스트 요청')
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select t_id, t_hideuser, t_hidedate from t_treasure where t_approval = 0"
    curs.execute(sql)
    data = curs.fetchall();
    print(data)
    data_list = []
    
    for obj in data:
        data_dic = {
            't_id' : obj[0],
            'hide_user' : obj[1],
            'hide_date' : obj[2]
        }
        data_list.append(data_dic)


    curs.close()
    conn.close()
    
    return render_template('tList.html',data_list=data_list)

# 유저 리스트
@app.route('/userList', methods = ['GET'])
def userList():
    print('유저 리스트 요청')
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select * from t_user"
    curs.execute(sql)
    data = curs.fetchall();
    print(data)
    data_list = []
    
    for obj in data:
        data_dic = {
            'user_id' : obj[0],
            'user_name' : obj[2],
            'user_phone' : obj[4],
            'user_email' : obj[5],
            'user_startdate' : obj[8],
            'user_black' : obj[12]
        }
        data_list.append(data_dic)

    curs.close()
    conn.close()
    
    return render_template('userList.html',data_list=data_list)

# 블랙 리스트
@app.route('/blackList', methods = ['GET'])
def blackList():
    print('블랙리스트 요청')
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select * from t_user where user_black = 0"
    curs.execute(sql)
    data = curs.fetchall();
    print(data)
    data_list = []
    
    for obj in data:
        data_dic = {
            'user_id' : obj[0],
            'user_name' : obj[2],
            'user_phone' : obj[4]
        }
        data_list.append(data_dic)

    curs.close()
    conn.close()
    
    return render_template('blackList.html',data_list=data_list)

#보물 승인
@app.route('/approveTreasure' , methods = ['GET'])
def approveTreasure():
    print('보물승인 요청')
    t_id = request.args['t_id']
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select t_hideuser, t_hidedate,t_latitude, t_longitude from t_treasure where t_id = "+str(t_id)
    curs.execute(sql)
    data = curs.fetchall()
    
    data_list = []
    
    for obj in data:
        data_dic = {
            't_hideuser' : obj[0],
            't_hidedate' : obj[1],
            't_latitude' : obj[2],
            't_longitude': obj[3]
        }
        data_list.append(data_dic)
    curs.close()
    conn.close()
    return render_template('checkTreasure.html', data_list = data_list)

#블랙리스트 on/off
@app.route('/applyBlack' , methods = ['GET'])
def applyBlack():
    print('블랙리스트 변경')
    user_id = request.args['user_id']
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = f"update t_user set user_black = 0 where user_id = '{user_id}'"
    curs.execute(sql)
    conn.commit()
    
    curs.close()
    conn.close()

    return redirect(url_for('userList'))

# 대쉬보드 
@app.route('/mainPage', methods = ['GET'])
def mainPage():
    print('보물갯수 요청')
    conn = db.connect(db_id, db_pw, url)
    curs=conn.cursor()
    sql = "select (select count(t_approval) from t_treasure where t_approval = 0) as hide,(select count(t_approval) from t_treasure where t_approval = 1) as find, (select count(user_id) from t_user) as uu from t_treasure t, t_user u  where rownum = 1"
    curs.execute(sql)
    data = curs.fetchall()
    
    data_list = []

    for obj in data:
        data_dic = {
            'hideTreasure' : obj[0],
            'findTreasure' : obj[1],
            'userCount' : obj[2]
        }
        data_list.append(data_dic)
    print(data_list)
    
    curs.close()
    conn.close()
    
    return render_template('index.html',data_list = data_list)
          
if __name__ == '__main__':
    app.run(host='192.168.35.90', port=5013)

 * Serving Flask app "__main__" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off


 * Running on http://192.168.21.8:5013/ (Press CTRL+C to quit)
192.168.20.156 - - [05/Sep/2022 17:51:39] "POST /login HTTP/1.1" 200 -


로그인 시도
[('pky5335', 'qwer1234!', '박광연', datetime.datetime(1991, 4, 13, 0, 0), '01053761493', 'pky5335@nate.com', '남', 1, datetime.datetime(2022, 9, 5, 11, 11, 25), None, None, 0, '박광연')]
pky5335 로그인
마커갱신
[(1, 6, 'cii', 'igo', 'igi', 'pky5335', None, 1, 35.1467274, 126.922126, datetime.datetime(2022, 9, 5, 11, 31, 3), 20), (3, 2, '제일큰', '나무밑', '사무', 'pky5335', None, 1, 35.146111, 126.922111, datetime.datetime(2022, 9, 5, 14, 11, 7), 20), (4, 2, '주차장', '가로등', '장난감', 'columbus', None, 1, 35.146222, 126.922333, datetime.datetime(2022, 9, 5, 14, 11, 54), 20), (5, 2, '대성학', '원3층', '1강실', 'pky5335', None, 1, 35.146999, 126.922999, datetime.datetime(2022, 9, 5, 14, 13), 20), (6, 2, '에어컨', '환풍기', '뒤', 'columbus', None, 1, 35.146555, 126.922555, datetime.datetime(2022, 9, 5, 14, 14, 4), 20), (7, 2, '공원', '제일큰', '나무밑', 'pky5335', None, 1, 35.145158, 126.923793, datetime.datetime(2022, 9, 5, 14, 15, 9), 20)]


192.168.20.156 - - [05/Sep/2022 17:51:40] "POST /mappage HTTP/1.1" 200 -
192.168.20.156 - - [05/Sep/2022 17:51:40] "POST /mappage HTTP/1.1" 200 -


갱신완료
1,6,cii,igo,igi,pky5335,None,1,35.1467274,126.922126,2022-09-05 11:31:03,20/3,2,제일큰,나무밑,사무,pky5335,None,1,35.146111,126.922111,2022-09-05 14:11:07,20/4,2,주차장,가로등,장난감,columbus,None,1,35.146222,126.922333,2022-09-05 14:11:54,20/5,2,대성학,원3층,1강실,pky5335,None,1,35.146999,126.922999,2022-09-05 14:13:00,20/6,2,에어컨,환풍기,뒤,columbus,None,1,35.146555,126.922555,2022-09-05 14:14:04,20/7,2,공원,제일큰,나무밑,pky5335,None,1,35.145158,126.923793,2022-09-05 14:15:09,20/
마커갱신
[(1, 6, 'cii', 'igo', 'igi', 'pky5335', None, 1, 35.1467274, 126.922126, datetime.datetime(2022, 9, 5, 11, 31, 3), 20), (3, 2, '제일큰', '나무밑', '사무', 'pky5335', None, 1, 35.146111, 126.922111, datetime.datetime(2022, 9, 5, 14, 11, 7), 20), (4, 2, '주차장', '가로등', '장난감', 'columbus', None, 1, 35.146222, 126.922333, datetime.datetime(2022, 9, 5, 14, 11, 54), 20), (5, 2, '대성학', '원3층', '1강실', 'pky5335', None, 1, 35.146999, 126.922999, datetime.datetime(2022, 9, 5, 14, 13), 20), (6, 2, '에어컨', '환풍기', '뒤', 'columbus', None, 1, 35.146555, 126.922555

...

마커갱신
[(1, 6, 'cii', 'igo', 'igi', 'pky5335', None, 1, 35.1467274, 126.922126, datetime.datetime(2022, 9, 5, 11, 31, 3), 26), (3, 2, '제일큰', '나무밑', '사무', 'pky5335', None, 1, 35.146111, 126.922111, datetime.datetime(2022, 9, 5, 14, 11, 7), 26), (4, 2, '주차장', '가로등', '장난감', 'columbus', None, 1, 35.146222, 126.922333, datetime.datetime(2022, 9, 5, 14, 11, 54), 26), (5, 2, '대성학', '원3층', '1강실', 'pky5335', None, 1, 35.146999, 126.922999, datetime.datetime(2022, 9, 5, 14, 13), 26), (6, 2, '에어컨', '환풍기', '뒤', 'columbus', None, 1, 35.146555, 126.922555, datetime.datetime(2022, 9, 5, 14, 14, 4), 26), (7, 2, '공원', '제일큰', '나무밑', 'pky5335', None, 1, 35.145158, 126.923793, datetime.datetime(2022, 9, 5, 14, 15, 9), 26)]
갱신완료
1,6,cii,igo,igi,pky5335,None,1,35.1467274,126.922126,2022-09-05 11:31:03,26/3,2,제일큰,나무밑,사무,pky5335,None,1,35.146111,126.922111,2022-09-05 14:11:07,26/4,2,주차장,가로등,장난감,columbus,None,1,35.146222,126.922333,2022-09-05 14:11:54,26/5,2,대성학,원3층,1강실,pky5335,None,1,35.146999,126.922999,2022-0

192.168.20.168 - - [05/Sep/2022 18:35:36] "POST /mappage HTTP/1.1" 200 -


마커갱신
[(1, 6, 'cii', 'igo', 'igi', 'pky5335', None, 1, 35.1467274, 126.922126, datetime.datetime(2022, 9, 5, 11, 31, 3), 26), (3, 2, '제일큰', '나무밑', '사무', 'pky5335', None, 1, 35.146111, 126.922111, datetime.datetime(2022, 9, 5, 14, 11, 7), 26), (4, 2, '주차장', '가로등', '장난감', 'columbus', None, 1, 35.146222, 126.922333, datetime.datetime(2022, 9, 5, 14, 11, 54), 26), (5, 2, '대성학', '원3층', '1강실', 'pky5335', None, 1, 35.146999, 126.922999, datetime.datetime(2022, 9, 5, 14, 13), 26), (6, 2, '에어컨', '환풍기', '뒤', 'columbus', None, 1, 35.146555, 126.922555, datetime.datetime(2022, 9, 5, 14, 14, 4), 26), (7, 2, '공원', '제일큰', '나무밑', 'pky5335', None, 1, 35.145158, 126.923793, datetime.datetime(2022, 9, 5, 14, 15, 9), 26)]
갱신완료
1,6,cii,igo,igi,pky5335,None,1,35.1467274,126.922126,2022-09-05 11:31:03,26/3,2,제일큰,나무밑,사무,pky5335,None,1,35.146111,126.922111,2022-09-05 14:11:07,26/4,2,주차장,가로등,장난감,columbus,None,1,35.146222,126.922333,2022-09-05 14:11:54,26/5,2,대성학,원3층,1강실,pky5335,None,1,35.146999,126.922999,2022-0

192.168.20.168 - - [05/Sep/2022 18:35:36] "POST /mappage HTTP/1.1" 200 -


마커갱신
[(1, 6, 'cii', 'igo', 'igi', 'pky5335', None, 1, 35.1467274, 126.922126, datetime.datetime(2022, 9, 5, 11, 31, 3), 26), (3, 2, '제일큰', '나무밑', '사무', 'pky5335', None, 1, 35.146111, 126.922111, datetime.datetime(2022, 9, 5, 14, 11, 7), 26), (4, 2, '주차장', '가로등', '장난감', 'columbus', None, 1, 35.146222, 126.922333, datetime.datetime(2022, 9, 5, 14, 11, 54), 26), (5, 2, '대성학', '원3층', '1강실', 'pky5335', None, 1, 35.146999, 126.922999, datetime.datetime(2022, 9, 5, 14, 13), 26), (6, 2, '에어컨', '환풍기', '뒤', 'columbus', None, 1, 35.146555, 126.922555, datetime.datetime(2022, 9, 5, 14, 14, 4), 26), (7, 2, '공원', '제일큰', '나무밑', 'pky5335', None, 1, 35.145158, 126.923793, datetime.datetime(2022, 9, 5, 14, 15, 9), 26)]
갱신완료
1,6,cii,igo,igi,pky5335,None,1,35.1467274,126.922126,2022-09-05 11:31:03,26/3,2,제일큰,나무밑,사무,pky5335,None,1,35.146111,126.922111,2022-09-05 14:11:07,26/4,2,주차장,가로등,장난감,columbus,None,1,35.146222,126.922333,2022-09-05 14:11:54,26/5,2,대성학,원3층,1강실,pky5335,None,1,35.146999,126.922999,2022-0

In [ ]:
!pip install cx_Oracle